# Feature selection

Na aula de hoje, vamos explorar os seguintes tópicos em Python:

- 1) Introdução
- 2) Lasso
- 3) Feature importance com árvores
- 4) RFE


In [1]:
import numpy  as np
import pandas as pd 

import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split

____
____
_____

## 1) Introdução

O processo de **feature selection** (**seleção de atributos**) consiste na escolha, com base em alguns critérios, de um **subconjunto do conjunto original** de features de um dado problema, que proporcionem um modelo com performance comparável ao modelo treinado com todas as features. 

<img src=https://miro.medium.com/max/694/0*D_jQ5yBsvCZjEYIW width=400>

O resultado do processo de feature selection é uma **redução na dimensionalidade** do espaço de features do problema (mas aqui, diferente do PCA, trabalhamos no espaço de features originais!)

Assim, o processo remove features redundantes ou irrelevantes. 

Dentre as vantagens do procedimento, podemos destacar:

- Maior eficiência no treinamento (afinal, reduzimos a quantidade de informação a ser processada);
- Eliminação de redundâncias (como multicolinearidade, por exemplo, que pode ser problemática para alguns estimadores);
- Um modelo mais enxuto, com menos features, é, em geral, mais facilmente interpretável;
- Ao reduzirmos o número de features, a complexidade da hipótese é reduzida, o que pode favorecer a generalização;

O princípio da [navalha de Occam](https://pt.wikipedia.org/wiki/Navalha_de_Ockham) é relevante no contexto de feature selection em projetos de machine learning. Sugiro [este post](https://machinelearningmastery.com/ensemble-learning-and-occams-razor/#:~:text=Occam's%20razor%20suggests%20that%20in,narrow%20and%20not%20generalize%20well.) para uma discussão deste princípio como uma heurística para a construção de modelos. Para uma discussão mais profunda, sugiro [este paper](https://www.aaai.org/Papers/KDD/1998/KDD98-006.pdf).



Na aula de hoje, veremos alguns procedimentos de feature selection. Vamos começar!

______________

## 2) LASSO

Já conhecemos um método capaz de realizar feature selection: a **regularização L1 (LASSO)**.

Diferente da regularização L2, quando utilizamos regularização L1 é possível zerar alguns dos parâmetros do modelo:

<img src=https://ugc.futurelearn.com/uploads/assets/2b/fe/2bfe399e-503e-4eae-9138-a3d7da738713.png width=800>

Embora ambas as modalidades de regularização tenha, sido introduzidas com o intuito de simplificar o espaço de hipóteses, o LASSO faz isso de maneira explítica, efetivamente possibilitando a realização de feature selection!

No entanto, há um problema: são poucos os métodos que têm o LASSO incorporado (ex.: regressão linear, logística, XGBoost).

Assim, se quisermos realizar feature selection utilizando outros estimadores, precisamos de técnicas mais genéricas, que é o que veremos a seguir.

Para utilizarmos o L1, uma abordagem possível é:

- **treinar inicialmente um modelo com LASSO**; 
- identificar quais features **ainda estão presentes no modelo** (isto é, com `coef_` não nulo);
- utilizar estas features apenas para treinar o estimador desejado.

Embora esta seja uma possibilidade, veremos, a seguir, que há técnicas que possibilitam este procedimento, de maneira mais geral!

______________

## 3) Feature importance com árvores

Além de estimadores poderosos, podemos utilizar árvores para fazer feature selection! 

Há duas formas comuns de utilizarmos árvores para a determinação da importância de features. Vamos conhecer cada uma na prática, utilizando um dataset familiar!

### `.feature_importances_`, com base em decréscimo de impureza (MDI)

Neste caso, o score de importância de cada uma das features é calculado com base na **média e desvio padrão da diminuição de impureza que cada feature proporciona na árvore (ou em cada árvore, no caso de ensembles)**.

O método é conhecido como **mean decrease in impurity** (MDI).

Este método é rápido, no entanto, o valor é fortemente enviesado para features que têm alta cardinalidade (features numéricas, ou features categóricas com muitos níveis).

Neste caso, é melhor utilizar o método de permutation feature importance. Para uma comparação detalhada entre os dois métodos, [veja esta página](https://scikit-learn.org/stable/auto_examples/inspection/plot_permutation_importance.html#sphx-glr-auto-examples-inspection-plot-permutation-importance-py).

Os resultados não são tão legais com uma única árvore. O método fica mais robusto se utilizarmos um ensemble:

Vamos dar uma olhada na variação...

A variação é enorme! O que pode ter acontecido?

Isso se deve justamente ao viés indesejado que é introduzido pelo MDI. Para corrigir isso, vamos introduzir um novo método!

### `permutation_importance()`, com base em permutação de features

Neste método, utilizamos a função `sklearn.inspection.permutation_importance()`, que vai criar permutações das features, mantendo um registro de um score, e como ele é afetado quando uma ou outra feature é eliminada.

Por realizar diversas permutações, este método é mais custoso, mas tem a vantagem de eliminar o viés que features de alta cardinalidade carregam com o método baseado em impureza.

Para maiores detalhes sobre o método, [clique aqui!](https://scikit-learn.org/stable/modules/permutation_importance.html#permutation-importance)

> Observação: embora tenhamos ilustrado a importância de permutação com a árvore de decisão, na realidade este é um método que pode ser usado com qualquer estimador!

A função que vamos utilizar [está aqui!](https://scikit-learn.org/stable/modules/generated/sklearn.inspection.permutation_importance.html)

A partir daqui, o próximo passo seria treinar um modelo normalmente (pipeline, gridsearch, etc.), mas apenas com essas features selecionadas (idealmente, uma combinação dessas heurísticas).

______________

## 4) RFE

Conheceremos agora o método **Recursive Feature Elimination** (RFE).

O RFE é um método que se utiliza de um estimador capaz de atribuir um score de **importância** a cada uma das features.

> Por exemplo, podemos olhar para os coeficientes de um modelo linear (`coef_`), ou então, para os scores de importância de features (`feature_importances_`).

O método então considera recursivamente **subconjuntos cada vez menores de features**, da seguinte maneira:

- O estimador é treinado inicialmente com todas as features;
- A importância de cada uma das features é calculada;
- As features menos importantes são retiradas do conjunto de features;
- O processo recomeça, até que o número  desejado de features seja alcançado.

Sendo assim, temos dois hiperparâmetros importantes na classe [RFE](https://scikit-learn.org/stable/modules/generated/sklearn.feature_selection.RFE.html):

- `estimator`: o estimador que irá disponibilizar os scores de importância de features;
- `n_features_to_select`: a quantidade de features que o subconjunto final terá.

Na prática, podemos utilizar um gridsearch para otimizar estes dois hiperparâmetros, ou então utilizar a classe [RFECV](https://scikit-learn.org/stable/modules/generated/sklearn.feature_selection.RFECV.html), que determina o melhor númeo de features automaticamente.

Vamos ver o método na prática!

Vamos ver como funciona o [RFECV](https://scikit-learn.org/stable/modules/generated/sklearn.feature_selection.RFECV.html):

Na prática, podemos incluir o RFE como um passo da Pipeline, e otimizar seus parâmetros com o grid search!

Vamos rodar o modelo completo, só pra ver se o modelo com menos 2 features é melhor

Vamos mudar o estimador do RFE - usamos a regressão logística com o LASSO!

Ou seja, estamos usando os `.coef_` pra atribuir a importância das features

_____________

Uma versão mais "enxuta" (tem mais variáveis auxilares pros objetos) do código com o grid search